In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import lxml.etree as ET
from geojson import Point, Feature, FeatureCollection, Polygon, dump
from openslide import open_slide

In [2]:
# Path to raw data 
# main_directory = os.path.dirname(os.getcwd())
# print(main_directory)
# os.chdir(main_directory + '/data/raw/xml_labels')
# xml_data_directory = os.getcwd()
# xml_files = os.listdir(xml_data_directory)
# # file name
# xml_file = xml_files[0]

/Users/alexandrasmith/Desktop/Workspace/Projects/masters


In [2]:
# get xml files for svs files
# Path to raw data 
main_directory = os.path.dirname(os.getcwd())

#  Read in svs file
os.chdir(main_directory + '/data/raw/svs_files')
svs_data_directory = os.getcwd()
svs_files = os.listdir(svs_data_directory)

In [3]:
svs_file = svs_files[2]
slide = open_slide(svs_file)
slide_props = slide.properties
image_id = slide_props["aperio.Filename"]

# Read in xml file
os.chdir(main_directory + '/data/raw/xml_labels')
xml_data_directory = os.getcwd()

xml_file = image_id + '.xml'
# Check working with correct svs corresponding to xml file
print(f'svs file name is {slide_props["aperio.Filename"]} and xml file name is {xml_file}')
print(f'Working with same files: {xml_file.rstrip(".xml") == slide_props["aperio.Filename"]}')

svs file name is TCGA-C8-A12Q-01Z-00-DX1 and xml file name is TCGA-C8-A12Q-01Z-00-DX1.xml
Working with same files: True


In [7]:
points = []
features = []
save_destination = '/Users/alexandrasmith/Desktop/Workspace/Projects/masters/data/interim/geojson_labels/'
file_id = xml_file.rstrip('.xml')
# Get annotations from XML file
tree = ET.parse(xml_file)
root = tree.getroot()

for Annotation in root.findall("./Annotation"): # for all annotations
    # iterate on all regions
    for Region in Annotation.findall("./*/Region"):
        # extract region attributes
        regionID = Region.attrib['Id']; regionArea = Region.attrib['Area']
        # iterate all points in region
        for Vertex in Region.findall("./*/Vertex"):
            # get points
            x_point = float(np.int32(np.float64(Vertex.attrib['X'])))
            y_point = float(np.int32(np.float64(Vertex.attrib['Y'])))
            points.append([x_point, y_point])
        # Ensure that polygon produced is in closed form 
        # i.e having same start and end point
        starting_point = points[0]
        if starting_point != points[-1]:
            points.append(starting_point)
        features.append(Feature(geometry=Polygon([points]), properties={"name": xml_file, "region_id": regionID, "object_type": "annotation"}))

with open(save_destination + file_id + '.geojson', 'w') as f:
    dump(features, f)